<h2> Análisis exploratorio: Bay Area Bike Share</h2>
<div> 
    <img src="http://www.bayareabikeshare.com/assets/images/bayarea/pages/pricing/bike_pricing.png">
</div>

In [ ]:
import pandas as pd
import plotly
from plotly.offline import iplot, init_notebook_mode
init_notebook_mode()

import plotly.plotly as py
import plotly.graph_objs as go
import numpy

In [ ]:
# Cargamos el data frame del registro de viajes
trips_df = pd.read_csv('../input/trip.csv')

# Cargamos el data frame del registro climático
weather_df = pd.read_csv('../input/weather.csv')

# Cargamos el data frame del registro de estaciones
stations_df = pd.read_csv('../input/station.csv')

In [ ]:
# Convertimos el campo "start_date" de string a datetime
trips_df['start_date'] = pd.to_datetime(trips_df['start_date'])

# Agregamos 3 columnas más con los siguientes datos de incio de los viajes: Fecha, hora, año y día de la semana
trips_df['start_date_only'] = trips_df['start_date'].dt.date
trips_df['start_hour'] = trips_df['start_date'].dt.hour
trips_df['start_year'] = trips_df['start_date'].dt.year
trips_df['start_weekday'] = trips_df['start_date'].dt.dayofweek

# Elimino viajes que duren más de un día (86400 segundos)
trips_df = trips_df[trips_df['duration'] < 86400]

In [ ]:
# Convertimos el campo "date" de string a date
weather_df['date'] = pd.to_datetime(weather_df['date']).dt.date

# Eliminamos filas inválidas
weather_df = weather_df.dropna(subset=['mean_temperature_f', 'mean_humidity', 'mean_wind_speed_mph', 'cloud_cover', 'precipitation_inches', 'events'], how='any')
weather_df = weather_df[weather_df['precipitation_inches'] != 'T']

# Convertimos el campo "precipitacion_inches" a float
weather_df['precipitation_inches'] = weather_df['precipitation_inches'].astype(float)

<h2> Análisis inicial </h2>

In [ ]:
# Cantidad de viajes registrados
len(trips_df)

In [ ]:
# Ejemplo de viaje
trips_df.sample()

In [ ]:
# Tipos de datos de trips_df
trips_df.dtypes

In [ ]:
# Agregamos una columna en trips_df con el nombre del día de la semana en el que se inició el viaje
dias_de_la_semana = {
    0: 'Lunes',
    1: 'Martes',
    2: 'Miércoles',
    3: 'Jueves',
    4: 'Viernes',
    5: 'Sábado',
    6: 'Domingo'
}

trips_df['start_weekday_name'] = trips_df['start_weekday'].apply(lambda numero: dias_de_la_semana[numero])
trips_counts_by_day = trips_df['start_weekday_name'].value_counts(sort=False)
trips_counts_by_day = trips_counts_by_day.reindex(['Lunes', 'Martes', 'Miércoles', 'Jueves', 'Viernes', 'Sábado', 'Domingo'])

In [ ]:
# Gráfico de cantidad de alquileres por día de la semana
trace = go.Bar(
    x=trips_counts_by_day.index,
    y=trips_counts_by_day.values,
    marker=dict(
        color='#f49e42',
        ),
    opacity=0.7
)

data = [trace]
layout = go.Layout(
    title='Cantidad de bicicletas alquiladas por día de la semana',
    titlefont=dict(
        family='Raleway',
        size=25
    ),
    xaxis=dict(
        title='Días de la semana',
        titlefont=dict(
            family='Raleway',
            size=16
        ),
        tickmode='array',
        tickvals=['Lunes', 'Martes', 'Miércoles', 'Jueves', 'Viernes', 'Sábado', 'Domingo']
    ),
    yaxis=dict(
        title='Cantidad de bicicletas alquiladas',
        titlefont=dict(
            family='Raleway',
            size=16
        ),
    )
)

figure = go.Figure(data=data, layout=layout)
plotly.offline.iplot(figure)

In [ ]:
# Gráfico de cantidad de alquileres por año
trace = go.Bar(
    x=trips_df['start_year'].value_counts().index,
    y=trips_df['start_year'].value_counts().values,
    marker=dict(
        color='#f49e42',
        ),
    opacity=0.7
)

data = [trace]
layout = go.Layout(
    title='Cantidad de bicicletas alquiladas por año',
    titlefont=dict(
        family='Raleway',
        size=25
    ),
    xaxis=dict(
        title='Año',
        titlefont=dict(
            family='Raleway',
            size=16
        ),
        tickmode='array',
        tickvals=[2013, 2014, 2015]
    ),
    yaxis=dict(
        title='Cantidad de bicicletas alquiladas',
        titlefont=dict(
            family='Raleway',
            size=16
        ),
    )
)

figure = go.Figure(data=data, layout=layout)
plotly.offline.iplot(figure, filename='Cantidad de bicicletas alquiladas por año')

In [ ]:
# Cantidad de registros climáticos
len(weather_df)

In [ ]:
# Ejemplo de registro climático
weather_df.sample()

In [ ]:
# Tipos de datos de weather_df
weather_df.dtypes

In [ ]:
# Tipos de eventos climáticos posibles
weather_df.events.unique()

In [ ]:
# Normalizamos los eventos climáticos
weather_df['events'] = weather_df['events'].replace(['rain'],'Rain')
weather_df.events.unique()

In [ ]:
# Cantidad de estaciones
stations_df['id'].unique()

In [ ]:
# Ejemplo
stations_df.sample()

In [ ]:
# Tipos de datos
stations_df.dtypes

<h2> Research questions </h2>

<h3> #1 ¿Podemos saber si el tránsito afecta a la duración de los viajes?</h3>

<p> Analizamos la cantidad de alquileres que se registran por hora del día. Vamos a suponer que las horas con más alquileres, son las horas pico de tránsito durante la semana. En cambio, durante el fin de semana, las horas con mayor cantidad de alquileres seguramente no tengan ningun tipo de relación con el tránsito </p>

In [ ]:
trace = go.Bar(
    x=trips_df['start_hour'].value_counts().index,
    y=trips_df['start_hour'].value_counts().values,
    marker=dict(
        color='rgb(247, 91, 120)',
        ),
    opacity=0.8
)

data = [trace]
layout = go.Layout(
    title='Cantidad de bicicletas alquiladas por hora',
    titlefont=dict(
        family='Raleway',
        size=25
    ),
    xaxis=dict(
        title='Horas del día',
        titlefont=dict(
            family='Raleway',
            size=16
        ),
        tickmode='array',
        tickvals=[0, 1, 2, 3, 4, 5, 
                  6, 7, 8, 9, 10, 11, 
                  12, 13, 14, 15, 16, 17, 
                  18, 19, 20, 21, 22, 23]
    ),
    yaxis=dict(
        title='Cantidad de bicicletas alquiladas',
        titlefont=dict(
            family='Raleway',
            size=16
        ),
    )
)

figure = go.Figure(data=data, layout=layout)
plotly.offline.iplot(figure, filename='Cantidad de bicicletas alquiladas por hora')

<p> ¿Que pasa los fines de semana? </p>

In [ ]:
# Horas con mayor cantidad de alquileres solo durante el fin de semana
trips_count_by_weekend_hour = trips_df[(trips_df.start_weekday_name == 'Sábado') | (trips_df.start_weekday_name == 'Domingo')]['start_hour'].value_counts()

In [ ]:
trace = go.Bar(
    x=trips_count_by_weekend_hour.index,
    y=trips_count_by_weekend_hour.values,
    marker=dict(
        color='#81aff9',
        ),
    opacity=0.8
)

data = [trace]
layout = go.Layout(
    title='Cantidad de bicicletas alquiladas por hora',
    titlefont=dict(
        family='Raleway',
        size=25
    ),
    xaxis=dict(
        title='Horas del día',
        titlefont=dict(
            family='Raleway',
            size=16
        ),
        tickmode='array',
        tickvals=[0, 1, 2, 3, 4, 5, 
                  6, 7, 8, 9, 10, 11, 
                  12, 13, 14, 15, 16, 17, 
                  18, 19, 20, 21, 22, 23]
    ),
    yaxis=dict(
        title='Cantidad de bicicletas alquiladas',
        titlefont=dict(
            family='Raleway',
            size=16
        ),
    )
)

figure = go.Figure(data=data, layout=layout)
plotly.offline.iplot(figure, filename='Cantidad de bicicletas alquiladas por hora del fin de semana')

<p> Vemos si los horarios que suponemos pico de tránsito, afectan a la duración de los viajes</p>

In [ ]:
# Elegimos un día de la semana y dos estaciones y nos quedamos solo con esos viajes
trips_duration_by_hour = trips_df[(trips_df.start_weekday == 3) & (trips_df.start_station_id == 69)  & (trips_df.end_station_id == 65)][['duration', 'start_hour']].groupby(['start_hour']).mean()

# Completamos los horarios que no tienen registros con ceros
trips_duration_by_hour_df = pd.DataFrame(trips_duration_by_hour.values, index=trips_duration_by_hour.index, columns=['duration'])
trips_duration_by_hour_df.loc[4] = 0
trips_duration_by_hour_df = trips_duration_by_hour_df.sort_index()

In [ ]:
trace = go.Scatter(
    x = trips_duration_by_hour_df.index,
    y = trips_duration_by_hour_df['duration'].values,
    mode = 'lines+markers'
)

data = [trace]
layout = go.Layout(
    title='Promedio de duración de los viajes por hora',
    titlefont=dict(
        family='Raleway',
        size=25
    ),
    xaxis=dict(
        title='Horas del día',
        titlefont=dict(
            family='Raleway',
            size=16
        ),
        tickmode='array',
        tickvals=[0, 1, 2, 3, 4, 5, 
                  6, 7, 8, 9, 10, 11, 
                  12, 13, 14, 15, 16, 17, 
                  18, 19, 20, 21, 22, 23]
    ),
    yaxis=dict(
        title='Promedio de duración (segundos)',
        titlefont=dict(
            family='Raleway',
            size=16
        ),
    )
)

fig = dict(data=data, layout=layout)
plotly.offline.iplot(fig, filename='Promedio de duración por hora')

<p> Se ve un pico de mayor duración a las 8 AM y otro a las 18 PM que estarían dentro de los supuestos horarios mas tránsitados. A su vez, se ven picos altos a las 2 AM y a las 3 AM, que seguramente no tengan que ver con el tránsito </p>

<h3> #2 ¿Cómo afecta el clima a los viajes? </h3>

<p> 2.1 ¿Afecta a la cantidad de viajes?</p>

In [ ]:
# Nos quedamos solo con los viajes realizados los días lunes
trips_counts_by_date = trips_df[trips_df.start_weekday == 0]['start_date_only'].value_counts()

In [ ]:
trips_counts_by_date_df = pd.DataFrame(trips_counts_by_date.values, index=trips_counts_by_date.index, columns=['Bicicletas alquiladas'])

In [ ]:
# Nos quedamos solo con las métricas climáticas que consideramos relevantes
some_metrics_of_weather_df = weather_df[['date', 'mean_temperature_f', 'mean_humidity', 'mean_wind_speed_mph', 'cloud_cover', 'precipitation_inches']].groupby(['date']).mean()
weather_by_date_df = pd.DataFrame(some_metrics_of_weather_df.values, index=some_metrics_of_weather_df.index, columns=['Temperatura promedio(F)', 'Humedad promedio', 'Velocidad del viento promedio(mph)', 'Nubosidad', 'Precipitacion(inches)'])

In [ ]:
# Creamos un nuevo data frame con algunos datos climáticos de cada día
count_and_weather_by_date_df = pd.merge(trips_counts_by_date_df, weather_by_date_df, left_index=True, right_index=True)
count_and_weather_by_date_df = count_and_weather_by_date_df.sort_values(by='Bicicletas alquiladas', ascending=False)

In [ ]:
# Ejemplo del nuevo data frame
count_and_weather_by_date_df.sample()

In [ ]:
# Temperatura promedio por cantidad de alquileres (solo en los días lunes)
trace = go.Scatter(
    y = count_and_weather_by_date_df['Temperatura promedio(F)'].values,
    x = count_and_weather_by_date_df['Bicicletas alquiladas'].values,
    mode = 'markers',
    marker=dict(
        size='12',
        color = '#F64E8B'
    )
)

layout= go.Layout(
    title='Temperatura promedio por número de alquileres diarios',
    titlefont=dict(
        family='Raleway',
        size=25
    ),
    xaxis= dict(
        title= 'Bicicletas alquiladas',
        titlefont=dict(
            family='Raleway',
            size=16
        ),
    ),
    yaxis=dict(
        title= 'Temperatura promedio (F)',
        titlefont=dict(
            family='Raleway',
            size=16
        ),
    )
)
fig= go.Figure(data=[trace], layout=layout)
plotly.offline.iplot(fig)

In [ ]:
# Nubosidad por cantidad de alquileres (solo en los días lunes)
count_and_weather_by_date_df['Nubosidad'] = count_and_weather_by_date_df['Nubosidad'].apply(lambda x: round(x, 0))
count_by_cloud_cover = count_and_weather_by_date_df[['Bicicletas alquiladas', 'Nubosidad']].groupby(['Nubosidad']).sum()

trace = go.Bar(
    y = count_by_cloud_cover['Bicicletas alquiladas'].values,
    x = count_by_cloud_cover['Bicicletas alquiladas'].index.tolist(),
    marker=dict(
        color='#3A4750',
        ),
    opacity=0.8
)

layout= go.Layout(
    title='Nubosidad por número de alquileres diarios',
    titlefont=dict(
            family='Raleway',
            size=25
        ),
    xaxis= dict(
        title= 'Nubosidad',
        titlefont=dict(
            family='Raleway',
            size=16
        ),
    ),
    yaxis=dict(
        title= 'Bicicletas alquiladas',
        titlefont=dict(
            family='Raleway',
            size=16
        ),
    )
)
fig= go.Figure(data=[trace], layout=layout)
plotly.offline.iplot(fig)

In [ ]:
# Humedad promedio por cantidad de alquileres (solo en los días lunes)
count_and_weather_by_date_df['Humedad promedio'] = count_and_weather_by_date_df['Humedad promedio'].apply(lambda x: round(x, 0))
count_by_humidity = count_and_weather_by_date_df[['Bicicletas alquiladas', 'Humedad promedio']].groupby(['Humedad promedio']).sum()

trace = go.Bar(
    y = count_by_humidity['Bicicletas alquiladas'].values,
    x = count_by_humidity['Bicicletas alquiladas'].index.tolist(),
    marker=dict(
        color='#c994c7',
        ),
    opacity=0.8
)

layout= go.Layout(
    title='Humedad promedio por número de alquileres diarios',
    titlefont=dict(
            family='Raleway',
            size=25
        ),
    xaxis= dict(
        title= 'Humedad promedio',
        titlefont=dict(
            family='Raleway',
            size=16
        ),
        tickmode='array',
        tickvals=count_by_humidity['Bicicletas alquiladas'].index.tolist()
    ),
    yaxis=dict(
        title= 'Bicicletas alquiladas',
        titlefont=dict(
            family='Raleway',
            size=16
        ),
    )
)
fig= go.Figure(data=[trace], layout=layout)
plotly.offline.iplot(fig)

In [ ]:
# Precipitación por cantidad de alquileres (solo en los días lunes)
dates_with_more_rents = go.Scatter(
    y = count_and_weather_by_date_df['Precipitacion(inches)'].head(10).values,
    x = count_and_weather_by_date_df['Bicicletas alquiladas'].head(10).values,
    mode='markers',
    marker=dict(
        size='12',
        color = '#F64E8B'
    ),
    name='En días con más alquileres'
)

dates_with_lees_rents = go.Scatter(
    y = count_and_weather_by_date_df['Precipitacion(inches)'].tail(10).values,
    x = count_and_weather_by_date_df['Bicicletas alquiladas'].tail(10).values,
    mode='markers',
    marker=dict(
        size='12',
        color = '#3A4750'
    ),
    name='En días con menos alquileres'
)

layout= go.Layout(
    title='Precipitación por número de alquileres diarios',
    titlefont=dict(
            family='Raleway',
            size=25
        ),
    xaxis= dict(
        title= 'Bicicletas alquiladas',
        titlefont=dict(
            family='Raleway',
            size=16
        ),
    ),
    yaxis=dict(
        title= 'Precipitación(inches)',
        titlefont=dict(
            family='Raleway',
            size=16
        ),
    )
)
fig= go.Figure(data=[dates_with_more_rents, dates_with_lees_rents], layout=layout)
plotly.offline.iplot(fig)

<p> 2.2 ¿Afecta a la duración de los viajes?</p>

In [ ]:
# Nos quedamos con los viajes realizados los días miércoles a las 17 horas entre las estaciones 66 y 72
duration_by_date = trips_df[(trips_df.start_station_id == 69) & (trips_df.end_station_id == 65) & (trips_df.start_weekday == 1) & (trips_df.start_hour == 17)][['duration', 'start_date_only']]
duration_by_date = duration_by_date.groupby(['start_date_only']).mean()

In [ ]:
# Creamos un nuevo data frame con la duración promedio de los viajes filtrados por fecha
trips_duration_by_date_df = pd.DataFrame(duration_by_date.values, index=duration_by_date.index, columns=['Duración promedio'])

In [ ]:
# Mergeamos el data frame anterior con el data frame con registros climáticos creado anteriormente
duration_and_weather_by_date_df = pd.merge(trips_duration_by_date_df, weather_by_date_df, left_index=True, right_index=True)

In [ ]:
# Ordenamos por promedio de duración
duration_and_weather_by_date_df = duration_and_weather_by_date_df.sort_values(by='Duración promedio', ascending=False)

In [ ]:
duration_and_weather_by_date_df

<p> Se puede ver que para la fecha y las estaciones elegidas tanto los tiempos como las variables climáticas son muy similares </p>

<h3> #3 ¿Cuáles son las estaciones más populares?</h3>

In [ ]:
# Estacion de origen más popular
trips_df['start_station_id'].value_counts().index[0]

In [ ]:
# Cantidad de viajes iniciados por estación
trace = go.Bar(
    x=trips_df['start_station_id'].value_counts().index,
    y=trips_df['start_station_id'].value_counts().values,
    marker=dict(
        color='#addd8e',
        ),
    opacity=0.8
)

data = [trace]
layout = go.Layout(
    title='Cantidad de viajes iniciados por estación',
    titlefont=dict(
        family='Raleway',
        size=25
    ),
    xaxis=dict(
        title='Estaciones',
        titlefont=dict(
            family='Raleway',
            size=16
        ),
        tickmode='array',
        tickvals=stations_df['id'].unique(),
        type="category"
    ),
    yaxis=dict(
        title='Cantidad de viajes iniciados',
        titlefont=dict(
            family='Raleway',
            size=16
        ),
    )
)

figure = go.Figure(data=data, layout=layout)
plotly.offline.iplot(figure, filename='Cantidad de viajes iniciados por estación')

In [ ]:
# Estacion de destino más popular
trips_df['end_station_id'].value_counts().index[0]

In [ ]:
# Cantidad de viajes finalizados por estación
trace = go.Bar(
    x=trips_df['end_station_id'].value_counts().index,
    y=trips_df['end_station_id'].value_counts().values,
    marker=dict(
        color='#fc8d59',
        ),
    opacity=0.8
)

data = [trace]
layout = go.Layout(
    title='Cantidad de viajes finalizados por estación',
    titlefont=dict(
        family='Raleway',
        size=25
    ),
    xaxis=dict(
        title='Estaciones',
        titlefont=dict(
            family='Raleway',
            size=16
        ),
        tickmode='array',
        tickvals=stations_df['id'].unique(),
        type="category"
    ),
    yaxis=dict(
        title='Cantidad de viajes finalizados',
        titlefont=dict(
            family='Raleway',
            size=16
        ),
    )
)

figure = go.Figure(data=data, layout=layout)
plotly.offline.iplot(figure, filename='Cantidad de viajes finalizados por estación')

In [ ]:
# Definimos el concepto de popularidad de una estación como:
# Viajes iniciados en la estación + Viajes finalizados en la estación / viajes totales * 2

total_trips = trips_df.shape[0]
def popularity(started_trips, ended_trips):
    popularity_percentage = (float(started_trips + ended_trips) / float(total_trips * 2)) * 100
    return round(popularity_percentage, 1)

stations = stations_df['id'].unique()
popularity_by_station = {}
for station_id in stations:
    started_trips = trips_df[trips_df.start_station_id == station_id].shape[0]
    ended_trips = trips_df[trips_df.end_station_id == station_id].shape[0]
    popularity_by_station[station_id] = popularity(started_trips, ended_trips)

stations_df['popularity'] = stations_df['id'].map(popularity_by_station)

In [ ]:
# Porcentaje de popularidad
trace = go.Bar(
    x=stations_df['id'].unique(),
    y=stations_df['popularity'].values,
    marker=dict(
        color='#bcbddc',
        ),
    opacity=0.8
)

data = [trace]
layout = go.Layout(
    title='Porcentaje de popularidad por estación',
    titlefont=dict(
        family='Raleway',
        size=25
    ),
    xaxis=dict(
        title='Estaciones',
        titlefont=dict(
            family='Raleway',
            size=16
        ),
        tickmode='array',
        tickvals=stations_df['id'].unique(),
        type='category'
    ),
    yaxis=dict(
        title='Porcentaje de popularidad',
        titlefont=dict(
            family='Raleway',
            size=16
        ),
    )
)

figure = go.Figure(data=data, layout=layout)
plotly.offline.iplot(figure, filename='Porcentaje de popularidad por estación')